#### ReportLab use guide

https://www.reportlab.com/docs/reportlab-userguide.pdf

#### ReportLab source code

https://bitbucket.org/rptlab/reportlab/src/998faa6891c038a2453a7828df2be53f7b68ce1d/src/reportlab/?at=default

#### Notes

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

https://pairlist2.pair.net/pipermail/reportlab-users/2005-June/004070.html

* Very useful code snippet: 

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

* Reportlab: Mixing Fixed Content and Flowables

https://www.blog.pythonlibrary.org/2012/06/27/reportlab-mixing-fixed-content-and-flowables/



## Approach

Approaches to generating calendar pages:

a) Don't use page templates, use just canvas and page breaks

It's possible to draw table on canvas! That's good.

b) Use templates and implement custom flowables to allow custom drawing in the pages

https://www.blog.pythonlibrary.org/2014/03/10/reportlab-how-to-create-custom-flowables/

c) Not sure is possible: use templates and somehow draw on canvas on each page.

Try to somehow access the canvas while creating the story. Maybe some get_canvas method.

d) Use page templates and use global variable to change what's on each page - 

```
def on_page3(canvas, doc):
    global page_data
    canvas.saveState()
    canvas.line(20,20, -20,20)
    canvas.restoreState()
    canvas.drawString(100, 600, f"String on page template 3:{page_data}")
```

I will probably have to use a combination of frames and canvas, because I need tables and tables are flowables. So maybe put a table tightly into a frame. 

## Notes
#### Grid instead of table:
canas.grid(), page 16 of ReportLab manual 

Advantage: it's not flowable, can place it exactly
Disadvantage: how do I put string in the cells? ... tedious

In [91]:
from reportlab.lib.units import inch, cm, mm
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, B5
from reportlab.platypus import SimpleDocTemplate, BaseDocTemplate, PageTemplate, Frame, Spacer, Table, \
TableStyle, Paragraph, NextPageTemplate
from reportlab.graphics.shapes import Drawing
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfgen import canvas
from reportlab.lib.utils import isSeq, encode_label, decode_label, annotateException, strTypes
from reportlab.platypus import Table as pdfTable

import calendar
import datetime

import os
import logging
import copy
from collections import namedtuple

In [2]:
import reportlab


In [3]:
logger = logging.Logger('root', level=logging.DEBUG)
logging.basicConfig()

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter.datefmt = '%m/%d/%Y %I:%M:%S %p' #03/11/2018 11:54:48 AM

#console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

logger.addHandler(console_handler)


In [4]:
period = {
    "calendar_start_date" :  
    "calendar_end_date" :
}

SyntaxError: invalid syntax (<ipython-input-4-327063e00bdb>, line 3)

In [4]:
options = {
    'pagesize' : reportlab.lib.pagesizes.B5,
    'margins' : tuple(4*[0.5*cm]) #left, right, upper, lower
    #'font' : "Courier-Bold"
}
options

{'margins': (14.173228346456693,
  14.173228346456693,
  14.173228346456693,
  14.173228346456693),
 'pagesize': (500.3149606299212, 708.6614173228346)}

In [5]:
content = {
    'start_month_for_months_section' : 7,
    'end_month_for_months_section' : 8
}
assert( content['start_month_for_months_section'] <= content['end_month_for_months_section'] )

In [6]:
height = options['pagesize'][1]
width = options['pagesize'][0]

In [7]:
cal = [['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']]
cal.extend(calendar.monthcalendar(2018,5))

In [17]:
class Counters():
    def __init__(self):
        self.page = 0
        self.week = 0


In [18]:
counters = Counters()

## Try drawing canvas page by page

## Try a month page

In [31]:
days_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

def get_month(year, month):
    
    cal = calendar.Calendar()
    month_data = []
    for d in cal.itermonthdays2(year, month):
        #print(d[0], days_names[d[1]])
        if d[0]!=0:
            cell = str(d[0])+(len(str(d[0]))==1)*" "+" "+days_names[d[1]]
            month_data.append(cell)

    month_data = [ [i, ""] for i in month_data]
    return month_data

if 1:
    print( get_month(2018, 5) )

[['1  Tue', ''], ['2  Wed', ''], ['3  Thu', ''], ['4  Fri', ''], ['5  Sat', ''], ['6  Sun', ''], ['7  Mon', ''], ['8  Tue', ''], ['9  Wed', ''], ['10 Thu', ''], ['11 Fri', ''], ['12 Sat', ''], ['13 Sun', ''], ['14 Mon', ''], ['15 Tue', ''], ['16 Wed', ''], ['17 Thu', ''], ['18 Fri', ''], ['19 Sat', ''], ['20 Sun', ''], ['21 Mon', ''], ['22 Tue', ''], ['23 Wed', ''], ['24 Thu', ''], ['25 Fri', ''], ['26 Sat', ''], ['27 Sun', ''], ['28 Mon', ''], ['29 Tue', ''], ['30 Wed', ''], ['31 Thu', '']]


In [32]:
def footer(canvas, s):
    canvas.drawString(width/2, 0, s)

In [24]:
def gen_months_page(options, first_month, second_month, mycanvas):
    
    logger.info(f"Generating months {first_month} and {second_month}")
    mycanvas.drawString(0,height-10,"Months section")

    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    mycanvas.drawString(10,height-20,calendar.month_name[first_month])

    month_data = get_month(2018,first_month)
    
    table = Table(month_data, colWidths=[20*mm,46*mm]) # altogether 66mm

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 10, 50)

    # ------- second month on a page
    
    month_data = get_month(2018,second_month)

    mycanvas.drawString(250,height-50,calendar.month_name[second_month])

    table = Table(month_data, colWidths=[50,150])

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h2 = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 250, 50+h-h2) # use previous height h so that they are aligned at the top 
    
    footer(mycanvas, str(1))
    mycanvas.showPage() 
    
if 1:
    c = canvas.Canvas('test.pdf', pagesize = options['pagesize'])
    gen_months_page(options, 5, 6, c)
    c.save()
    os.system("start " + 'test.pdf')

05/13/2018 02:18:36 PM - root - INFO - Generating months 5 and 6


In [37]:
def gen_months_section(options, content, canvas):
    
    start_month = content['start_month_for_months_section']
    last_month = content['end_month_for_months_section']
    
    assert(start_month<last_month)
    
    if (last_month-start_month+1) % 2 == 0: #even number of months
        logger.info("Even number of months.")
        pass
    else: # odd number of months, add one month
        logger.debug("Even number of months.")
        last_month += 1 # to-do: handle spilling over a year
    
    for m in range(start_month, last_month, 2):
        gen_months_page(options, m, m+1, canvas)
    
if 1:
    logger.setLevel(logging.DEBUG)
    c = canvas.Canvas('test.pdf', pagesize = options['pagesize'])
    
    test_options = options
    test_content = { 'start_month_for_months_section': 5,
                     'end_month_for_months_section': 11
                   }

    gen_months_section(test_options, test_content, c)
    c.save()
    os.system("start " + 'test.pdf')    
    

05/13/2018 02:20:04 PM - root - DEBUG - Even number of months.
05/13/2018 02:20:04 PM - root - INFO - Generating months 5 and 6
05/13/2018 02:20:04 PM - root - INFO - Generating months 7 and 8
05/13/2018 02:20:04 PM - root - INFO - Generating months 9 and 10
05/13/2018 02:20:04 PM - root - INFO - Generating months 11 and 12


In [33]:
cal = calendar.Calendar()
    
for i in calendar.Calendar().iterweekdays():
    print(i, calendar.day_name[i])

0 Monday
1 Tuesday
2 Wednesday
3 Thursday
4 Friday
5 Saturday
6 Sunday


In [26]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
    

c:\users\d\appdata\local\programs\python\python36-32\python.exe
3.6.1 (v3.6.1:69c0db5, Mar 21 2017, 17:54:52) [MSC v.1900 32 bit (Intel)]
sys.version_info(major=3, minor=6, micro=1, releaselevel='final', serial=0)


In [57]:
cal = calendar.Calendar()
cal.monthdatescalendar(2018, 5)

[[datetime.date(2018, 4, 30),
  datetime.date(2018, 5, 1),
  datetime.date(2018, 5, 2),
  datetime.date(2018, 5, 3),
  datetime.date(2018, 5, 4),
  datetime.date(2018, 5, 5),
  datetime.date(2018, 5, 6)],
 [datetime.date(2018, 5, 7),
  datetime.date(2018, 5, 8),
  datetime.date(2018, 5, 9),
  datetime.date(2018, 5, 10),
  datetime.date(2018, 5, 11),
  datetime.date(2018, 5, 12),
  datetime.date(2018, 5, 13)],
 [datetime.date(2018, 5, 14),
  datetime.date(2018, 5, 15),
  datetime.date(2018, 5, 16),
  datetime.date(2018, 5, 17),
  datetime.date(2018, 5, 18),
  datetime.date(2018, 5, 19),
  datetime.date(2018, 5, 20)],
 [datetime.date(2018, 5, 21),
  datetime.date(2018, 5, 22),
  datetime.date(2018, 5, 23),
  datetime.date(2018, 5, 24),
  datetime.date(2018, 5, 25),
  datetime.date(2018, 5, 26),
  datetime.date(2018, 5, 27)],
 [datetime.date(2018, 5, 28),
  datetime.date(2018, 5, 29),
  datetime.date(2018, 5, 30),
  datetime.date(2018, 5, 31),
  datetime.date(2018, 6, 1),
  datetime.date(

In [70]:
week

[datetime.date(2018, 5, 28),
 datetime.date(2018, 5, 29),
 datetime.date(2018, 5, 30),
 datetime.date(2018, 5, 31),
 datetime.date(2018, 6, 1),
 datetime.date(2018, 6, 2),
 datetime.date(2018, 6, 3)]

In [83]:
week[0].day_name()

AttributeError: 'datetime.date' object has no attribute 'day_name'

In [82]:
week[1].weekday()

1

In [104]:
d = datetime.date(2018, 4, 30)

In [121]:
d.day


30

In [111]:
d.weekday()

0

In [108]:
calendar.day_name[0]

'Monday'

In [138]:
def gen_week(options, content, week, mycanvas):
    ''' Generate a single week
    
    Current design is 1 week on 2 pages.
    
    week ... list of days in the week, first is Monday, last is Sunday
    [datetime.date(2018, 4, 30), datetime.date(2018, 5, 1), datetime.date(2018, 5, 2), 
     datetime.date(2018, 5, 3), datetime.date(2018, 5, 4), datetime.date(2018, 5, 5), datetime.date(2018, 5, 6)]
    
    '''
    
    logger.debug(week)
    
    
    week_section_options = {
        'top_margin' : 9*mm,
        'bottom_margin' : 9*mm,
        'top_box_height' : 3*cm,
        'weekly_tasks_width' : 5*cm,
        'weekly_tasks_row_height' : 6*mm,
        'weekly_focus_height' : 5*cm,
        'day_box_width' : 7*cm,
        'day_box_nlines' : 4, # number of lines inside each day box 
    }
    
    
    mycanvas.drawString(0,height-10,f"Week section, week={week[0]}-{week[-1]}")
    
    # boxes for days of the week
    day_box_height =( height-week_section_options['top_margin']-week_section_options['bottom_margin'] ) / 7
        
    # spacing of lines inside each day box
    day_box_line_spacing = day_box_height / (week_section_options['day_box_nlines']+1)
    logger.debug(day_box_line_spacing)
        
    for i, day in enumerate( week ):
        day_name = calendar.day_name[day.weekday()]
        logger.debug(f"{day}:{day_name}")
        mycanvas.grid([week_section_options['weekly_tasks_width'], 
                       width],
                      [height-week_section_options['top_margin']-i*day_box_height, 
                       height-week_section_options['top_margin']-(i+1)*day_box_height])
        # name of the day in the day box
        mycanvas.drawString(week_section_options['weekly_tasks_width'], 
                            height-week_section_options['top_margin']-i*day_box_height-(1)*day_box_line_spacing,
                            f"{day_name[0:2]}:{day.day}.{day.month}")

                                                 
        mycanvas.setStrokeColor('pink')
        for j in range( week_section_options['day_box_nlines'] ):
            mycanvas.line(week_section_options['weekly_tasks_width'],
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          week_section_options['weekly_tasks_width'] + (width-week_section_options['weekly_tasks_width'])/2-0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
            mycanvas.line(week_section_options['weekly_tasks_width']+(width-week_section_options['weekly_tasks_width'])/2+0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          width,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
        mycanvas.setStrokeColor('black')
                      
    
    # top box
    
    table = Table([['1']], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['top_box_height'])
    
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, height-week_section_options['top_box_height'])
    
    
    # left column with week tasks
    
    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    
    nr = int( (height-week_section_options['top_box_height']-week_section_options['weekly_focus_height'])//(6*mm) )

    table = Table(data=[ [i] for i in range(int(nr))], 
                  colWidths=[5*cm], rowHeights=nr*[week_section_options['weekly_tasks_row_height']]) 
    
    table.setStyle(TableStyle(common_style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table height is {h}")
    table.drawOn(mycanvas, 0, week_section_options['weekly_focus_height'])
    
    
    # weekly focus subsection
    table = Table([['1']], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['weekly_focus_height'])
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0),
          (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, 0)
    
    footer(mycanvas, str(1))
    
    # complete page
    mycanvas.showPage() 
    
    
    
    return canvas
    
    

if 1:
    
    logger.setLevel(logging.DEBUG)
    c = canvas.Canvas('test.pdf', pagesize = options['pagesize'])
    week = calendar.Calendar().monthdatescalendar(2018,5)[0]
    gen_week(options, content, week, c)
    c.save()
    os.system("start " + 'test.pdf') 

05/16/2018 11:33:43 PM - root - DEBUG - [datetime.date(2018, 4, 30), datetime.date(2018, 5, 1), datetime.date(2018, 5, 2), datetime.date(2018, 5, 3), datetime.date(2018, 5, 4), datetime.date(2018, 5, 5), datetime.date(2018, 5, 6)]
05/16/2018 11:33:43 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:33:43 PM - root - DEBUG - 2018-04-30:Monday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-01:Tuesday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-02:Wednesday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-03:Thursday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-04:Friday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-05:Saturday
05/16/2018 11:33:43 PM - root - DEBUG - 2018-05-06:Sunday
05/16/2018 11:33:43 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:33:43 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:33:43 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)


In [137]:
def gen_weeks(options, content, canvas):
    
    
    logger.info("Generating weeks")
    canvas.drawString(10,500,"Weeks section")
    
    for month in range(content['start_month_for_weeks_section'], content['end_month_for_weeks_section']+1):
    
        cal = calendar.Calendar()
        weeks_in_month = cal.monthdatescalendar(2018, month)

        for w, week in enumerate(weeks_in_month):
            print("week ", w)
            gen_week(options, content, week, canvas)

            for day in week:
                print(day)

        # increase counter
        counters.week += 1


if 1:
    logger.setLevel(logging.DEBUG)
    c = canvas.Canvas('test.pdf', pagesize = options['pagesize'])
    
    content['start_month_for_weeks_section'] = 5
    content['end_month_for_weeks_section'] = 6
    
    gen_weeks(options, content, c)

    c.save()
    os.system("start " + 'test.pdf') 
    


05/16/2018 11:32:09 PM - root - INFO - Generating weeks
05/16/2018 11:32:09 PM - root - DEBUG - [datetime.date(2018, 1, 1), datetime.date(2018, 1, 2), datetime.date(2018, 1, 3), datetime.date(2018, 1, 4), datetime.date(2018, 1, 5), datetime.date(2018, 1, 6), datetime.date(2018, 1, 7)]
05/16/2018 11:32:09 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-01:Monday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-02:Tuesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-03:Wednesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-04:Thursday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-05:Friday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-06:Saturday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-01-07:Sunday
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:32:09 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (1

week  0
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
week  1
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
week  2
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
week  3
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
week  4
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-03
2018-02-04
week  0
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-03
2018-02-04
week  1
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
week  2
2018-02-12
2018-02-13
2018-02-14

05/16/2018 11:32:09 PM - root - DEBUG - [datetime.date(2018, 2, 19), datetime.date(2018, 2, 20), datetime.date(2018, 2, 21), datetime.date(2018, 2, 22), datetime.date(2018, 2, 23), datetime.date(2018, 2, 24), datetime.date(2018, 2, 25)]
05/16/2018 11:32:09 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-19:Monday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-20:Tuesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-21:Wednesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-22:Thursday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-23:Friday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-24:Saturday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-02-25:Sunday
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:32:09 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
05/16/2018


2018-02-15
2018-02-16
2018-02-17
2018-02-18
week  3
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
week  4
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-03
2018-03-04
week  0
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-03
2018-03-04
week  1
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
week  2
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-17
2018-03-18
week  3
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
week  4
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01
week  0
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01
week  1


05/16/2018 11:32:09 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-02:Monday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-03:Tuesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-04:Wednesday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-05:Thursday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-06:Friday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-07:Saturday
05/16/2018 11:32:09 PM - root - DEBUG - 2018-04-08:Sunday
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:32:09 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:32:09 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
05/16/2018 11:32:09 PM - root - DEBUG - [datetime.date(2018, 4, 9), datetime.date(2018, 4, 10), datetime.date(2018, 4, 11), datetime.date(2018, 4, 12), datetime.date(2018, 4, 13), datetime.date(2018, 4, 14), datetime.date(2018, 4, 15)]
05/16/2018 

...

2018-08-27
2018-08-28
2018-08-29
2018-08-30
2018-08-31
2018-09-01
2018-09-02
week  1
2018-09-03
2018-09-04
2018-09-05
2018-09-06
2018-09-07
2018-09-08
2018-09-09
week  2
2018-09-10
2018-09-11
2018-09-12
2018-09-13
2018-09-14
2018-09-15
2018-09-16
week  3
2018-09-17
2018-09-18
2018-09-19
2018-09-20
2018-09-21
2018-09-22
2018-09-23
week  4
2018-09-24
2018-09-25
2018-09-26
2018-09-27
2018-09-28
2018-09-29
2018-09-30
week  0
2018-10-01
2018-10-02
2018-10-03
2018-10-04
2018-10-05
2018-10-06
2018-10-07
week  1
2018-10-08
2018-10-09
2018-10-10
2018-10-11
2018-10-12
2018-10-13
2018-10-14
week  2


05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-21:Sunday
05/16/2018 11:32:10 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:32:10 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:32:10 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
05/16/2018 11:32:10 PM - root - DEBUG - [datetime.date(2018, 10, 22), datetime.date(2018, 10, 23), datetime.date(2018, 10, 24), datetime.date(2018, 10, 25), datetime.date(2018, 10, 26), datetime.date(2018, 10, 27), datetime.date(2018, 10, 28)]
05/16/2018 11:32:10 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-22:Monday
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-23:Tuesday
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-24:Wednesday
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-25:Thursday
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-26:Friday
05/16/2018 11:32:10 PM - root - DEBUG - 2018-10-27:Saturday
05/

2018-10-15
2018-10-16
2018-10-17
2018-10-18
2018-10-19
2018-10-20
2018-10-21
week  3
2018-10-22
2018-10-23
2018-10-24
2018-10-25
2018-10-26
2018-10-27
2018-10-28
week  4
2018-10-29
2018-10-30
2018-10-31
2018-11-01
2018-11-02
2018-11-03
2018-11-04
week  0
2018-10-29
2018-10-30
2018-10-31
2018-11-01
2018-11-02
2018-11-03
2018-11-04
week  1
2018-11-05
2018-11-06
2018-11-07
2018-11-08
2018-11-09
2018-11-10
2018-11-11
week  2
2018-11-12
2018-11-13
2018-11-14
2018-11-15
2018-11-16
2018-11-17
2018-11-18
week  3
2018-11-19
2018-11-20
2018-11-21
2018-11-22
2018-11-23
2018-11-24
2018-11-25
week  4


05/16/2018 11:32:11 PM - root - DEBUG - [datetime.date(2018, 11, 26), datetime.date(2018, 11, 27), datetime.date(2018, 11, 28), datetime.date(2018, 11, 29), datetime.date(2018, 11, 30), datetime.date(2018, 12, 1), datetime.date(2018, 12, 2)]
05/16/2018 11:32:11 PM - root - DEBUG - 18.7896512935883
05/16/2018 11:32:11 PM - root - DEBUG - 2018-11-26:Monday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-11-27:Tuesday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-11-28:Wednesday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-11-29:Thursday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-11-30:Friday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-12-01:Saturday
05/16/2018 11:32:11 PM - root - DEBUG - 2018-12-02:Sunday
05/16/2018 11:32:11 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
05/16/2018 11:32:11 PM - root - DEBUG - table height is 476.22047244094495
05/16/2018 11:32:11 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
05/16

2018-11-26
2018-11-27
2018-11-28
2018-11-29
2018-11-30
2018-12-01
2018-12-02
week  0
2018-11-26
2018-11-27
2018-11-28
2018-11-29
2018-11-30
2018-12-01
2018-12-02
week  1
2018-12-03
2018-12-04
2018-12-05
2018-12-06
2018-12-07
2018-12-08
2018-12-09
week  2
2018-12-10
2018-12-11
2018-12-12
2018-12-13
2018-12-14
2018-12-15
2018-12-16
week  3
2018-12-17
2018-12-18
2018-12-19
2018-12-20
2018-12-21
2018-12-22
2018-12-23
week  4
2018-12-24
2018-12-25
2018-12-26
2018-12-27
2018-12-28
2018-12-29
2018-12-30
week  5
2018-12-31
2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06


In [41]:
def gen_notes(options, content, canvas):
    logger.info("Generating notes section")
    canvas.drawString(0,height-30,"debug:Notes section")
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray, None, (1,4)),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        #('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        #('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        #('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        #('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]
    
    # make as many rows as columns as to fill the page
    row_size = 9*mm
    col_size = 9*mm
    
    rows = int( (height-30) // row_size )
    logger.debug(f"Rows:{rows}")
    
    horizontal_margin = 30
    
    cols = int( (width-2*horizontal_margin) // col_size )
    logger.debug(f"Cols:{cols}")
    
    table_data = [ ['' for i in range(cols)] for i in range(rows) ]
    table = Table(table_data, colWidths=cols*[col_size], rowHeights=rows*[row_size])
    logger.debug(f"{rows}, {cols}")
    table.setStyle(TableStyle(style))
    
    w, h = table.wrapOn(canvas, 0, 0)
    table.drawOn(canvas, horizontal_margin, 0)
    
    canvas.showPage() 

if 1:
    logger.setLevel(logging.DEBUG)
    c = canvas.Canvas('test.pdf', pagesize = options['pagesize'])

    gen_notes(test_options, test_content, c)
    
    c.save()
    os.system("start " + 'test.pdf')    

05/13/2018 02:20:41 PM - root - INFO - Generating notes section
05/13/2018 02:20:41 PM - root - DEBUG - Rows:26
05/13/2018 02:20:41 PM - root - DEBUG - Cols:17
05/13/2018 02:20:41 PM - root - DEBUG - 26, 17


In [35]:
# diary structure as a mapping pages->sections
# Is generated from diary_structure_template
# diary_structure_template -> diary_sections

diary_structure_template = ['months_v1'] + 2*['notes_v1'] + ['weeks_v1'] + 1*['notes_v1']

In [40]:
c = canvas.Canvas('diary.pdf', pagesize = options['pagesize'])

section_generators = {
    'months_v1' : gen_months_section,
    'weeks_v1' : gen_weeks,
    'notes_v1' : gen_notes
}

for section in diary_structure_template:
    logger.debug(section)
    section_generators[section](options, content, c) # calls a generator as per dictionary
    
c.save()

os.system("start " + 'diary.pdf')

05/13/2018 02:20:32 PM - root - DEBUG - months_v1
05/13/2018 02:20:32 PM - root - INFO - Even number of months.
05/13/2018 02:20:32 PM - root - INFO - Generating months 7 and 8
05/13/2018 02:20:32 PM - root - DEBUG - notes_v1
05/13/2018 02:20:32 PM - root - INFO - Generating notes section
05/13/2018 02:20:32 PM - root - DEBUG - Rows:26
05/13/2018 02:20:32 PM - root - DEBUG - Cols:17
05/13/2018 02:20:32 PM - root - DEBUG - 26, 17
05/13/2018 02:20:32 PM - root - DEBUG - notes_v1
05/13/2018 02:20:32 PM - root - INFO - Generating notes section
05/13/2018 02:20:32 PM - root - DEBUG - Rows:26
05/13/2018 02:20:32 PM - root - DEBUG - Cols:17
05/13/2018 02:20:32 PM - root - DEBUG - 26, 17
05/13/2018 02:20:32 PM - root - DEBUG - weeks_v1
05/13/2018 02:20:32 PM - root - INFO - Generating weeks
05/13/2018 02:20:32 PM - root - DEBUG - notes_v1
05/13/2018 02:20:32 PM - root - INFO - Generating notes section
05/13/2018 02:20:32 PM - root - DEBUG - Rows:26
05/13/2018 02:20:32 PM - root - DEBUG - Cols:

0

## Frames

In [ ]:
frame = Frame(options['margins'][0], # left margin
              options['margins'][3], # bottom margin
              0.5*(options['pagesize'][0]-options['margins'][2]-options['margins'][3]), # up to top margin
              0.5*(options['pagesize'][1]-options['margins'][0]-options['margins'][1]), # right to right margin
              id='f_normal', 
              showBoundary=True)
frame2 = Frame(0.5*(options['pagesize'][1]-options['margins'][0]-options['margins'][1]), # left margin
              options['margins'][3], # bottom margin
              0.5*(options['pagesize'][0]-options['margins'][2]-options['margins'][3]), # up to top margin
              0.5*(options['pagesize'][1]-options['margins'][0]-options['margins'][1]), # right to right margin
              id='f2_normal', 
              showBoundary=True)

frame_widht = 150
frame_height = 150

frames = []
f = Frame(0,300,frame_widht,frame_widht,
              showBoundary=True,)
frames.append(f)
frames.append(Frame(frame_widht,300,frame_widht,frame_height,
              showBoundary=True))
frames.append(Frame(0,150,frame_widht,frame_height,
              showBoundary=True))
frames.append(Frame(frame_widht,150,frame_widht,frame_height,
              showBoundary=True))


def on_page1(canvas, doc):
    canvas.saveState()
    canvas.line(10,10, 20,20)
    canvas.restoreState()
    table.drawOn(canvas, 50,50)
    canvas.drawString(100, 600, f"String on page template 1")

def on_page2(canvas, doc):
    canvas.saveState()
    canvas.line(20,20, -20,20)
    canvas.restoreState()
    canvas.drawString(100, 600, f"String on page template 2")
    
def on_page3(canvas, doc):
    global page_data
    canvas.saveState()
    canvas.line(20,20, -20,20)
    canvas.restoreState()
    canvas.drawString(100, 600, f"String on page template 3:{page_data}")
    
page_templates = []
page_templates.append( PageTemplate(id='page1',frames=frames, onPage=on_page1) )
global page_data
page_data = 'aaa'
page_templates.append( PageTemplate(id='page2',frames=frames, onPage=on_page2) )
page_templates.append( PageTemplate(id='page3',frames=frames, onPage=on_page3) )


In [ ]:
def get_story():
    story = []
    #story.append(NextPageTemplate('page1')) # set page template
    #story.append(Paragraph("Hello, world.", getSampleStyleSheet()['Normal']))

    # insert table
    #story.append(table)

    #story.append( reportlab.platypus.FrameBreak())
    # no need for breaking into next frame - it seems to jump to the first by default
    story.append(Paragraph("Hello, world 1.", getSampleStyleSheet()['Normal']))
    story.append(Paragraph("Hello, world 2.", getSampleStyleSheet()['Normal']))

    story.append( reportlab.platypus.FrameBreak())
    story.append(Paragraph("Hello, world in the second frame.", getSampleStyleSheet()['Normal']))

    story.append( reportlab.platypus.FrameBreak())
    story.append(Paragraph("Hello, world in the third frame.", getSampleStyleSheet()['Normal']))
    
    story.append( reportlab.platypus.FrameBreak())
    story.append(Paragraph("Hello, world in the fourth frame.", getSampleStyleSheet()['Normal']))
    story.append(table)
    
    for i in range(2):
        
        # choose next page template
        story.append(NextPageTemplate('page3')) # set page template

        # next page
        story.append(reportlab.platypus.PageBreak())
        

        # again, jumps to the first frame automatically
        story.append(Paragraph(f"Hello, world 1{i}.", getSampleStyleSheet()['Normal']))
        story.append(Paragraph("Hello, world 2.", getSampleStyleSheet()['Normal']))

        story.append( reportlab.platypus.FrameBreak())
        story.append(Paragraph("Hello, world in the second frame.", getSampleStyleSheet()['Normal']))

    return story   

In [ ]:
doc = BaseDocTemplate('calendar.pdf', 
                        pagesize=options['pagesize'],
                        pageTemplates=page_templates)
doc.build(get_story())
os.system("start " + 'calendar.pdf')

In [ ]:
os.system("start " + 'calendar.pdf')